In [48]:
!pip install torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 5.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [12]:
import pydub
import librosa
import soundfile
import nemo.collections.asr as nemo_asr
import glob
import nemo.collections.tts as nemo_tts
import numpy as np
import torch
import shutil
import pyctcdecode
from word_beam_search import WordBeamSearch
from ctcdecode import CTCBeamDecoder
import itertools
import symspellpy
import matplotlib.pyplot as plt
from dataclasses import dataclass
import torchaudio
from forced_alignment import get_trellis, backtrack, plot_trellis_with_path, merge_repeats, plot_trellis_with_segments, merge_words
from IPython.display import Audio
import os

In [10]:
asr = nemo_asr.models.EncDecCTCModel.from_pretrained('QuartzNet15x5Base-En', strict=False).eval().cpu()
config =asr.to_config_dict()
labels = config['decoder']['vocabulary'] + ['|']

[NeMo I 2022-12-13 05:44:33 cloud:56] Found existing object /home/nikita/.cache/torch/NeMo/NeMo_1.13.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.
[NeMo I 2022-12-13 05:44:33 cloud:62] Re-using file from: /home/nikita/.cache/torch/NeMo/NeMo_1.13.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo
[NeMo I 2022-12-13 05:44:33 common:911] Instantiating model from pre-trained checkpoint
[NeMo I 2022-12-13 05:44:33 features:225] PADDING: 16
[NeMo I 2022-12-13 05:44:34 save_restore_connector:243] Model EncDecCTCModel was successfully restored from /home/nikita/.cache/torch/NeMo/NeMo_1.13.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.


In [36]:
emissions = asr.transcribe(['./test/furcation_18_grade_one.wav'], logprobs=True)[0]
transcript = asr.transcribe(['./test/furcation_18_grade_one.wav'], logprobs=False)[0]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [36]:
!youtube-dl https://www.youtube.com/watch\?v=8gv8LPfKDPM

[youtube] 8gv8LPfKDPM: Downloading webpage
[youtube] 8gv8LPfKDPM: Downloading MPD manifest
[dashsegments] Total fragments: 46
[download] Destination: WHAT IS DENTAL PLAQUE DENTAL PLAQUE MADE EASY - COMPOSITION - FORMATION - EFFECTS-8gv8LPfKDPM.f137.mp4
[download] 100% of 7.02MiB in 00:32.11MiB/s ETA 00:0004799
[download] Destination: WHAT IS DENTAL PLAQUE DENTAL PLAQUE MADE EASY - COMPOSITION - FORMATION - EFFECTS-8gv8LPfKDPM.f140.m4a
[download] 100% of 3.61MiB in 01:2607KiB/s ETA 00:00
[ffmpeg] Merging formats into "WHAT IS DENTAL PLAQUE DENTAL PLAQUE MADE EASY - COMPOSITION - FORMATION - EFFECTS-8gv8LPfKDPM.mp4"
Deleting original file WHAT IS DENTAL PLAQUE DENTAL PLAQUE MADE EASY - COMPOSITION - FORMATION - EFFECTS-8gv8LPfKDPM.f137.mp4 (pass -k to keep)
Deleting original file WHAT IS DENTAL PLAQUE DENTAL PLAQUE MADE EASY - COMPOSITION - FORMATION - EFFECTS-8gv8LPfKDPM.f140.m4a (pass -k to keep)


In [38]:
!ffmpeg -y -i *8gv8LPfKDPM.mp4 -vn  -ac 1 -ar 16000  audioclips/8gv8LPfKDPM.wav
#!rm *uiDzZZoWkOI.mkv

ffmpeg version n5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 12.2.0 (GCC)
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libdav1d --enable-libdrm --enable-libfreetype --enable-libfribidi --enable-libgsm --enable-libiec61883 --enable-libjack --enable-libmfx --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librav1e --enable-librsvg --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libv4l2 --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxcb --enable-libxml2 -

In [4]:
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
model = bundle.get_model().to('cpu')
labels = bundle.get_labels()

In [41]:
sm = symspellpy.SymSpell(max_dictionary_edit_distance=3)
grammar_words = ['gingival', 'gingivitis', 'gingiva', 'mesial', 'dental', 'formation', 'lingual', 'distal', 'bleeding', 'furcation', 'plaque', 'margin', 'severe']

for w in grammar_words:
    sm.create_dictionary_entry(w, 1)

In [105]:
y, sr  = librosa.load("./audio.wav", sr=16_000)

In [42]:
for audio_file in glob.glob("./audioclips/*.wav"):
    seg = pydub.AudioSegment.from_file(audio_file)
    filename = audio_file.split('/')[-1].split('.')[0]
    segs = pydub.silence.split_on_silence(seg, min_silence_len=1000, silence_thresh=-40)
    try:
        os.mkdir("./tmp/")
    except: 
        print("exists")
    for i, seg in enumerate(segs):
        pydub.AudioSegment.export(seg, f"tmp/{i}.wav", format='wav')

    for f in glob.glob('tmp/*.wav')[1:]:
        y, _ = librosa.load(f, sr=16_000)
        emissions, _ = model(torch.Tensor([y]))
        transcript = [labels[i] for i in np.argmax(emissions[0].detach().numpy(), axis=1)]
        transcript = [k for (k, g) in itertools.groupby(transcript)]
        transcript = ''.join(transcript).replace('-','')
        tmp = [sm.lookup_compound(w, 2)[0].term.upper() if len(sm.lookup(w.lower(), 2)) else w for w in transcript.split('|')]
        transcript = "|".join(tmp)
        if len(transcript.split('|')) == 0:
            continue
        dictionary = {c: i for i, c in enumerate(labels)}
        tokens = [dictionary[c] for c in transcript]
        trellis = get_trellis(emissions[0], tokens)
        path = backtrack(trellis, emissions[0], tokens)
        segments = merge_repeats(path, transcript)
        word_segments = merge_words(segments)
        for word in word_segments:
            for grammar_word in grammar_words:
                if word.label == grammar_word.upper():
                    print(word)
                    try:
                        os.mkdir(f"./train_samples/{grammar_word}")
                    except: 
                        print("exists")
                    i = len(glob.glob(f'./train_samples/{grammar_word}/{filename}_*'))
                    l = len(y)
                    start = word.start/emissions.shape[1]
                    end = word.end/emissions.shape[1]
                    st = int(start*l)
                    end =int(end*l)
                    soundfile.write(f"./train_samples/{grammar_word}/{filename}_{i}.wav", y[st:end], samplerate=16_000)
    shutil.rmtree('./tmp')

GINGIVITIS	(54995.76): [   14,    54)
GINGIVA	(75292.91): [   82,   104)
GINGIVITIS	(85867.85): [  167,   195)
exists
SEVERE	(1838426.86): [  383,   399)
GINGIVA	(556408.21): [  207,   231)
exists
GINGIVA	(818133.34): [  136,   157)
exists
GINGIVA	(678912.28): [   29,    48)
exists
GINGIVA	(374747.96): [  221,   245)
exists
GINGIVA	(590621.99): [  330,   349)
exists
GINGIVAL	(290998.05): [  402,   424)
GINGIVAL	(260536.46): [  481,   499)
exists
GINGIVAL	(133404.75): [   58,    79)
exists
GINGIVAL	(11795.49): [    3,    22)
exists
GINGIVITIS	(63633.22): [    4,    32)
exists
PLAQUE	(127830.40): [   95,   114)
GINGIVA	(152100.84): [  152,   178)
exists
DENTAL	(6044.02): [  233,   246)
DENTAL	(16869.33): [   11,    22)
exists
GINGIVA	(252638.79): [   58,    76)
exists
GINGIVA	(382746.74): [  270,   294)
exists
DENTAL	(359069.16): [  122,   139)
exists
GINGIVAL	(119539.36): [  359,   378)
exists
GINGIVAL	(94611.49): [  430,   449)
exists
GINGIVITIS	(27403.76): [   51,    78)
exists
GINGIV